In [15]:
import sys
import pandas as pd
import os
import platform
print(platform.python_version())

3.6.4


In [9]:
rootDir = 'V:/Dropbox/ShareIrene/ScienceFair/Experiments/DataFiles/20180222_1507'

In [16]:
motion_map = {'SwipeLeft': 1, 'SwipeRight': 2}

# Process csv file

1. Extract quaternions
2. Transpose and name data column

In [17]:
def prepare_csv_df(csv_df, dataName, motion):
    #1. Extract quaternions
    df = csv_df[['quat_w','quat_x','quat_y','quat_z']]
    #2. Transpose (stack)
    df = pd.DataFrame(df.stack())
    #3. Name data column (multi-index)
    columns = [(motion,dataName)]
    df.columns = pd.MultiIndex.from_tuples(columns)
    #4. Transpose
    df = df.T
    df.index.names = ['motion', 'dataName']
    return df

In [18]:
# all_df = pd.DataFrame()
# df_list = []
def addCsvDf(df_list, csv_df, dataName, motion):
    df = prepare_csv_df(csv_df, dataName, motion)
    if df.isnull().values.any():
        print("Null values")
    df_list.append(prepare_csv_df(csv_df, dataName, motion))
#     return none

# Loop through data directories
Assume that in the root directory is a set of sub-directories.<br>
The name of the sub-directory matches a motion-name.<br>
Each sub-directory contains a set of csv file, each with one motion.
One motion is a fixed set of samples of the IMU.


In [19]:
# motions_dict = {}
def read_csv_files(rootDir):
    df_list = []
    for dirName, subdirList, fileList in os.walk(rootDir):
        motionName = dirName.replace(rootDir, '') #Feature name based on the dir name
        if len(motionName) > 0:
#             print('Motion: %s' % motionName)
            for fname in fileList:
                filename, file_extension = os.path.splitext(fname)
                csv_file = os.path.join(dirName, fname)
                fileSize = os.path.getsize(csv_file)
                dataName = fname.replace('.CSV', '')
#                 print('\t%s, size = %i, data = %s' % (fname, fileSize, dataName))
                if (fileSize > 0):
                    df = pd.read_csv(csv_file)
                    if len(df.index) >= 48:
                        df = df.head(48)
                        if not df.isnull().values.any():
                            addCsvDf(df_list, df, dataName, motionName)
                        else:
                            print('Found null values')
                    else :
                        print('Less than 49 samples: ' + fname)
    all_df = pd.concat(df_list)
    if all_df.isnull().values.any():
        print ("NULL")
    return all_df

In [20]:
all_df = read_csv_files(rootDir)
all_df.head(100)

ValueError: No objects to concatenate

In [21]:
X = all_df
df = all_df.copy()
# df.index.names = ['motion', 'dataName']
df = df.reset_index()
Y = df['motion']
Y.head(10)

NameError: name 'all_df' is not defined

In [22]:
X.head()

NameError: name 'X' is not defined

In [ ]:
X.isnull().values.any()

In [ ]:
Y.isnull().values.any()

# Label Encoding
We need to map the textual values of the predicted labels to a number.

In [ ]:
y_encoded = Y.map(motion_map)
y_encoded.head()

# Learn

In [ ]:
# split the data to training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
#gnb = GaussianNB()
rdf = RandomForestClassifier()
# y_pred = gnb.fit(X, Y).predict(X)
# print("Number of mislabeled points out of a total %d points : %d" % (X.shape[0],(Y != y_pred).sum()))
model = rdf.fit(X_train, y_train)
y_pred = model.predict(X_train)
print("Number of mislabeled points out of a total %d points : %d" % (X_train.shape[0],(y_train != y_pred).sum()))

In [ ]:
from sklearn.metrics import classification_report
### call pipeline.predict() on your X_test data to make a set of test predictions
y_prediction = model.predict( X_test )
### test your predictions using sklearn.classification_report()
report = classification_report( y_test, y_prediction )
### and print the report
print(report)